<a href="https://colab.research.google.com/github/thiagorayam/RNA/blob/main/Projeto_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

#MLP

class MLP(object):

#Parâmetros da rede 
    def __init__(self, eta = 0.1, epoch =10000, epsilon=0.01, alfa=0, i=13, j=3, k=3, l=0, nlayer=1):
        self.eta = eta # taxa de aprendizagem
        self.epoch = epoch # número máximo de épocas
        self.epsilon = epsilon # erro médio admissível
        self.alfa=alfa # fator do termo momentum
        self.i=i # número de neurônios na camada de entrada
        self.j=j # número de neurônios na camada j
        self.k=k # número de neurônios na camada k
        self.l=l # número de neurônios na camada l
        self.nlayer= nlayer #número de camadas intermediárias

#Inicializa os pesos (matriz Wji) e bias (vetor teta_j) da rede. j,i representam os números de neurônios da camada de saída e entrada    
    def inicialize(self): 

        self.w_ji=np.random.uniform(-1,1, [self.j,self.i+1])
        self.w_kj=np.random.uniform(-1,1, [self.k,self.j+1])

        if (self.nlayer>1):
          self.w_lk=np.random.uniform(-1,1, [self.l,self.k+1])

        return self

#Função de ativação utilizada - sigmoide
    def sigm(self, x):
        return 1/(1 + np.exp(-x))

#Derivada da função sigmoide
    def d_sigm(self, x):
        return x * (1.0 - x)

#Cálculo da saída da camada escondida e da saída      
    def feed_net(self, y_i):
        self.y_i=y_i #entrada da camada i
        self.y_j=self.sigm(np.dot(self.w_ji,np.concatenate((y_i,[1])))) #entrada da camada j, termo adicionado devido ao bias
        self.y_k=self.sigm(np.dot(self.w_kj,np.concatenate((self.y_j,[1])))) #entrada da camada K, termo adicionado devido ao bias

        if (self.nlayer>1):
          self.y_l=self.sigm(np.dot(self.w_lk,np.concatenate((self.y_k,[1])))) #entrada da camada L, termo adicionado devido ao bias
        
        return self

#Cálculo da saída da camada escondida e da saída 
    def back_propagation(self, error):
        
        if (self.nlayer>1):
          self.delta_l=error*self.d_sigm(self.y_l) #Gradiente local do erro na camada de saída
          self.delta_k=np.dot(self.w_lk.T,self.delta_l)*self.d_sigm(np.concatenate((self.y_k,[1])))#Gradiente local do erro na camada escondida
          self.delta_k=self.delta_k[0:self.k] # O último elemento representava apenas a relação do erro do bias, mas o bias não se interliga com a camada anterior.
          
        else:
          self.delta_k=error*self.d_sigm(self.y_k) #Gradiente local do erro na camada de saída
        
        self.delta_j=np.dot(self.w_kj.T,self.delta_k)*self.d_sigm(np.concatenate((self.y_j,[1]))) #Gradiente local do erro na camada escondida
        self.delta_j=self.delta_j[0:self.j] # O último elemento representava apenas a relação do erro do bias, mas o bias não se interliga com a camada anterior.  

        return self

#Atualização dos pesos.
    def fit(self, X, y):
        
        emed=1.01
        count=0 
        self.inicialize()
        deltaw_lk=0
        deltaw_kj=0
        deltaw_ji=0

        plt.axis([0, self.epoch, 0, 0.2])
        
        while (emed > self.epsilon and count < self.epoch):
         
         index = np.arange(X.shape[0])
         np.random.shuffle(index)
         X = X[index]
         y = y[index]

         emed=0
         
         for n in range (np.size(X, 0)):

           y_i=X[n,:]
           self.feed_net(y_i)      
           
           if (self.nlayer>1):
             error=y[n,:]-self.y_l
           else:
             error=y[n,:]-self.y_k
                       
           emed += np.sum(np.square(error))
           self.back_propagation(error) 

           if (self.nlayer>1):
             self.w_lk=self.w_lk+self.eta*self.delta_l[:, np.newaxis]*np.concatenate((self.y_k,[1])) + self.alfa*deltaw_lk
          
           self.w_kj=self.w_kj+self.eta*self.delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           self.w_ji=self.w_ji+self.eta*self.delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1]))+ self.alfa*deltaw_ji

           if (self.nlayer>1):
             deltaw_lk=self.eta*self.delta_l[:, np.newaxis]*np.concatenate((self.y_k,[1])) + self.alfa*deltaw_lk
           
           deltaw_kj=self.eta*self.delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           deltaw_ji=-self.eta*self.delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1])) + self.alfa*deltaw_ji
           
           self.feed_net(y_i)
          
         emed /= np.size(X, 0)
         self.emed=emed
         plt.scatter(count,emed)
         count += 1

        print(count)

        return self

    def predict(self, X):
        ypred=[]
        for n in range (np.size(X, 0)):
          y_i=X[n,:]
          self.feed_net(y_i)

          if (self.nlayer>1):
            ypred.append(self.y_l)
          else: 
            ypred.append(self.y_k)

        return np.array(ypred)

In [ ]:
def compute_accuracy(a, b): #a = pred , b = y
  hits = 0
  for i in range(a.shape[0]):
    linha_a = a[i]
    linha_b = b[i]
    c = np.sum(linha_a[:min(len(linha_a), len(linha_b))] == linha_b[:min(len(linha_a), len(linha_b))])
    if c == len(linha_a):
      hits = hits + 1
  acc = hits / a.shape[0]
  return acc

In [ ]:
# Classificação Wine Data

N=178
dim=13

X=np.zeros((N, dim))
y=np.zeros(N)
ref_arquivo = open("/content/sample_data/wine.data","r")
for j in range (N):
  linha = ref_arquivo.readline()
  valores=linha.split(',')
  y[j]=valores[0]
  for i in range(dim):
    X[j,i]=valores[i+1]

X=X/X.max(axis=0)

arr = np.zeros((len(y), 3))
for i in range(len(y)):
  c = int(y[i])-1
  arr[i][c] = 1
y=arr

train_rate=0.75

index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
y = y[index]
X_train=X[0:int(N*train_rate)]
X_test=X[0:int(N*(1-train_rate))]
y_train=y[0:int(N*train_rate)]
y_test=y[0:int(N*(1-train_rate))]

mlp = MLP(eta=0.2, epoch=500, epsilon=0.005, alfa=0.2, i=dim, j=10, k=3, l=2, nlayer=1)
mlp.fit(X_train,y_train)
ypred=np.around(mlp.predict(X_test))

In [ ]:
# Regressão - Geographical Original of Music Data Set

N=1059
dim=68

X=np.zeros((N, dim))
y=np.zeros((N,2))
ref_arquivo = open("/content/sample_data/default_features_1059_tracks.txt","r")
for j in range (N):
  linha = ref_arquivo.readline()
  valores=linha.split(',')
  y[j,0]=valores[68]
  y[j,1]=valores[69]
  for i in range(dim):
    X[j,i]=valores[i+1]

Xmax=X.max(axis=0)
X=X/Xmax
ymax=y.max(axis=0)
y=y/ymax

train_rate=0.75

index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
y = y[index]
X_train=X[0:int(N*train_rate)]
X_test=X[0:int(N*(1-train_rate))]
y_train=y[0:int(N*train_rate)]
y_test=y[0:int(N*(1-train_rate))]

mlp = MLP(eta=1, epoch=500, epsilon=0.005, alfa=0, i=68, j=34, k=2, l=2, nlayer=1)
mlp.fit(X_train,y_train)
ypred=mlp.predict(X_test)